In [1]:
from api.db import db

await db.init()

In [2]:
import polars as pl

items = pl.read_csv("../db/csvs/item.csv")

await db.insert(items, "item", pkey_cols={"id"}, return_cols={"id"})

In [5]:
res = await db.fetch(
    "SELECT * FROM item WHERE id = ANY(:ids) ORDER BY price LIMIT 10",
    ids=[83473, 348374],
)
print(res)

shape: (2, 3)
┌────────┬─────────┬──────────────┐
│ id     ┆ name    ┆ price        │
│ ---    ┆ ---     ┆ ---          │
│ i64    ┆ str     ┆ decimal[*,2] │
╞════════╪═════════╪══════════════╡
│ 348374 ┆ yogurt  ┆ 3.99         │
│ 83473  ┆ pumpkin ┆ 12.99        │
└────────┴─────────┴──────────────┘


In [8]:
ids = pl.Series("id", [83473, 348374])
names = pl.Series("name", ["onion", "bbq", "beans"])
attr_dict = {"ids": ids, "names": names}

query = """--sql
    SELECT * 
    FROM item 
    WHERE 
        id = ANY(:ids) OR
        name = ANY(:names) 
    ORDER BY price 
    LIMIT 10
    """

await db.fetch(
    query,
    **attr_dict,
)

id,name,price
i64,str,"decimal[*,2]"
1965488,"""onion""",2.99
348374,"""yogurt""",3.99
1870644,"""bbq""",6.99
83473,"""pumpkin""",12.99
6334,"""beans""",69.99
